In [1]:
!pip -q install -U pip
!pip -q install -U "click>=8.2.1"
!pip -q install pdfplumber openai
!pip -q install git+https://github.com/openai/whisper.git
!pip -q install edge-tts
!apt-get -y install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


# 1. Montar o Google Drive para acessar os PDFs 📂

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



# 2. Localizar e filtrar PDFs das obras de Freud 📖

In [3]:
import os, re
import pdfplumber

pdf_path = "/content/drive/MyDrive/"  # seus PDFs do Freud estão na raiz do MyDrive, conforme sua listagem

# Lista PDFs na pasta
all_pdfs = [f for f in os.listdir(pdf_path) if f.lower().endswith(".pdf")]

# Filtro por nome (ajuste as palavras-chave se quiser)
pattern = r"freud|obras completas|imago|histeria|sonhos|psicanal"
freud_pdfs = [f for f in all_pdfs if re.search(pattern, f.lower())]

print("\nQtd PDFs na pasta:", len(all_pdfs))
print("Qtd PDFs do Freud (filtrados):", len(freud_pdfs))
print("Exemplos (até 10):", freud_pdfs[:10])

if len(freud_pdfs) == 0:
    raise FileNotFoundError(
        "Não encontrei PDFs do Freud com o filtro atual. "
        "Ajuste o 'pattern' ou mova seus PDFs para uma pasta específica e atualize pdf_path."
    )



Qtd PDFs na pasta: 253
Qtd PDFs do Freud (filtrados): 12
Exemplos (até 10): ['1.4-OBRAS COMPLETAS - VOLUME II - ESTUDOS SOBRE HISTERIA.pdf', '2.2-PRIMEIRAS PUBLICACOES PSICANALITICAS - OBRAS COMPLETAS VOLUME III.pdf', '2.3-OBRAS COMPLETAS - VOLUME IV - A INTERPRETACAO DOS SONHOS.pdf', '2.4-OBRAS COMPLETAS VOLUME V A INTERPRETACAO DOS SONHOS E SOBRE OS SONHOS.pdf', '1. Freud. Obras completas Volume VII.pdf', 'FREUD, Sigmund. Obras Completas (Imago) - Vol. 06 (1901).pdf', 'FREUD, Sigmund. Obras Completas (Imago) - Vol. 08 (1905).pdf', 'FREUD, Sigmund. Obras Completas (Imago) - Vol. 09 (1906 - 1908).pdf', 'FREUD, Sigmund. Obras Completas (Imago) - Vol. 10 (1909).pdf', 'FREUD, Sigmund. Obras Completas (Imago) - Vol. 11 (1910).pdf']


# 3. Extrair texto dos PDFs (corpus)

In [4]:
corpus_parts = []
total_pages = 0
pages_with_text = 0

# Ajuste para testes: limite de páginas (None = tudo)
MAX_PAGES = 200  # ex.: 200 para teste

for fname in sorted(freud_pdfs):
    full = os.path.join(pdf_path, fname)
    try:
        with pdfplumber.open(full) as pdf:
            for page in pdf.pages:
                total_pages += 1
                text = page.extract_text()
                if text and text.strip():
                    pages_with_text += 1
                    corpus_parts.append(text)

                if MAX_PAGES is not None and total_pages >= MAX_PAGES:
                    break
    except Exception as e:
        print(f"[AVISO] Falha ao ler {fname}: {e}")

    if MAX_PAGES is not None and total_pages >= MAX_PAGES:
        break

corpus = "\n".join(corpus_parts)

print("\n--- Corpus ---")
print("Páginas lidas:", total_pages)
print("Páginas com texto extraído:", pages_with_text)
print("Tamanho do corpus (chars):", len(corpus))

if pages_with_text == 0:
    print("\n[ATENÇÃO] Nenhuma página retornou texto com pdfplumber.")
    print("Isso normalmente significa PDF escaneado (imagem). Nesse caso você precisará OCR (pytesseract).")


--- Corpus ---
Páginas lidas: 200
Páginas com texto extraído: 197
Tamanho do corpus (chars): 549446


# 4. Gravação de áudio 🎤

In [7]:
from IPython.display import HTML, display
from google.colab import output
from base64 import b64decode
import uuid, os

_last_audio_path = None

def _save_audio_js(data_url, out_webm="/content/request_audio.webm"):
    global _last_audio_path
    audio_bytes = b64decode(data_url.split(",")[1])
    with open(out_webm, "wb") as f:
        f.write(audio_bytes)
    _last_audio_path = out_webm
    return "OK"

cb_name = f"rec_{uuid.uuid4().hex}"
output.register_callback(cb_name, _save_audio_js)

display(HTML(f"""
<div style="display:flex; gap:12px; align-items:center;">
  <button id="startBtn" style="padding:10px 14px;">🎙️ Gravar</button>
  <button id="stopBtn" style="padding:10px 14px;" disabled>⏹️ Parar</button>
  <span id="status" style="font-family:monospace;">Pronto.</span>
</div>

<script>
let stream, recorder, chunks;

const startBtn = document.getElementById("startBtn");
const stopBtn  = document.getElementById("stopBtn");
const statusEl = document.getElementById("status");

function blobToDataURL(blob) {{
  return new Promise((resolve) => {{
    const reader = new FileReader();
    reader.onloadend = () => resolve(reader.result);
    reader.readAsDataURL(blob);
  }});
}}

startBtn.onclick = async () => {{
  try {{
    statusEl.textContent = "Pedindo permissão do microfone...";
    stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
    recorder = new MediaRecorder(stream);
    chunks = [];

    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.onstart = () => {{
      statusEl.textContent = "Gravando... (clique em Parar)";
      startBtn.disabled = true;
      stopBtn.disabled = false;
    }};
    recorder.onstop = async () => {{
      statusEl.textContent = "Processando áudio...";
      const blob = new Blob(chunks, {{ type: recorder.mimeType }});
      const dataUrl = await blobToDataURL(blob);

      await google.colab.kernel.invokeFunction("{cb_name}", [dataUrl], {{}});
      statusEl.textContent = "Áudio enviado. Rode a próxima célula.";

      stopBtn.disabled = true;
      startBtn.disabled = false;
      stream.getTracks().forEach(t => t.stop());
    }};

    recorder.start();
  }} catch (e) {{
    statusEl.textContent = "Erro ao acessar microfone: " + e;
    startBtn.disabled = false;
    stopBtn.disabled = true;
  }}
}};

stopBtn.onclick = () => {{
  if (recorder && recorder.state === "recording") recorder.stop();
}};
</script>
"""))

# 5. Transcrição com Whisper 🧠

In [12]:
import whisper
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("\nGPU disponível?", torch.cuda.is_available(), "| device:", device)

model = whisper.load_model("small").to(device)

result = model.transcribe(
    record_file,
    fp16=(device == "cuda"),
    language="pt"
)

transcription = result["text"].strip()
print("\nTranscrição:", transcription)


GPU disponível? True | device: cuda

Transcrição: Estou com ansiedade, sonhos recorrentes e sensação de culpa. Como isso se relaciona com conflitos inconscientes?


# 6. OpenAI (SDK atual) + resposta 💬

In [13]:
import os
os.environ["OPENAI_API_KEY"] = "SUA CHAVE AQUI"
import os
from openai import OpenAI
import re

# 🔐 Pegar chave da variável de ambiente
api_key = os.environ.get("OPENAI_API_KEY", "").strip()

# ❗ Verificação de segurança
if not api_key:
    raise ValueError(
        "OPENAI_API_KEY não encontrada.\n"
        "Defina a variável com:\n"
        "os.environ['OPENAI_API_KEY'] = 'SUA_CHAVE'\n"
        "ou use o Secrets do Colab."
    )

# 🔑 Criar cliente
client = OpenAI(api_key=api_key)

# Seleção simples de contexto (heurística) para não mandar corpus aleatório
def pick_context_simple(corpus_text: str, query: str, max_chars=6000):
    words = re.findall(r"\w+", query.lower())
    if not words:
        return corpus_text[:2000]

    hits = []
    current_len = 0

    # usa até 12 palavras mais informativas (aproximação simples)
    key_words = [w for w in words if len(w) >= 4][:12]

    for line in corpus_text.split("\n"):
        low = line.lower()
        score = sum(1 for w in key_words if w in low)
        if score >= 2:
            hits.append(line)
            current_len += len(line) + 1
            if current_len >= max_chars:
                break

    if hits:
        return "\n".join(hits)[:max_chars]

    return corpus_text[:2000]

contexto = pick_context_simple(corpus, transcription, max_chars=6000)

resp = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "system",
            "content": (
                "Você é um analista virtual inspirado em Freud. "
                "Use conceitos psicanalíticos freudianos e linguagem clara, útil e cuidadosa. "
                "Faça perguntas complementares quando necessário. "
                "Não invente citações com páginas; use referências gerais ao corpus quando fizer sentido."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Pergunta (transcrição do áudio): {transcription}\n\n"
                f"Trechos do corpus freudiano (para apoio):\n{contexto}"
            ),
        },
    ],
)

chatgpt_response = resp.output_text.strip()
print("\nResposta:\n", chatgpt_response)


Resposta:
 Sua experiência de ansiedade, sonhos recorrentes e sensação de culpa pode estar profundamente ligada a conflitos inconscientes, conceito central na teoria psicanalítica de Freud. Freud nos ensinou que o inconsciente abriga desejos, medos e lembranças reprimidas que, por não serem aceitos pela consciência, se manifestam simbolicamente por meio de sintomas, sonhos e sentimentos incômodos, como a culpa.

Por exemplo, em “A Interpretação dos Sonhos” (1900), Freud explica que os sonhos funcionam como “vias régias para o inconsciente”, permitindo a expressão disfarçada de desejos reprimidos e conflitos internos que, se fossem diretamente percebidos, provocariam angústia ou culpa. Os sonhos recorrentes sugerem que esses desejos ou conflitos não foram resolvidos, persistindo a necessidade do inconsciente de os comunicar à consciência.

A ansiedade, do ponto de vista freudiano, muitas vezes surge como um sinal da luta interna entre os desejos do id (instintos), as proibições do supe

# 6. Síntese de voz com edge-TTS 🔊

In [14]:
import edge_tts
from IPython.display import Audio, display

async def tts_edge(text, out_mp3="/content/response_audio.mp3", voice="pt-BR-FranciscaNeural"):
    communicate = edge_tts.Communicate(text=text, voice=voice)
    await communicate.save(out_mp3)
    return out_mp3

response_audio = await tts_edge(chatgpt_response)  # ✅ aqui é await, não asyncio.run
display(Audio(response_audio, autoplay=True))